In [ ]:
import os
for dirname, _, filenames in os.walk('/kaggle/input'):
    for filename in filenames:
        print(os.path.join(dirname, filename))

In [ ]:
# Octopus ML pakage - github.com/gershonc/octopus-ml
!pip install octopus-ml

In [ ]:
import warnings
warnings.simplefilter("ignore")
import seaborn as sns 
import matplotlib.pyplot as plt
import time
import pandas as pd
import numpy as np
import lightgbm as lgb
import tracemalloc
from pandas_summary import DataFrameSummary
from sklearn.metrics import classification_report
%matplotlib inline
sns.set_style("whitegrid")

pd.set_option('display.max_columns', None)  # or 1000
pd.set_option('display.max_rows', None)  # or 1000
pd.set_option('display.max_colwidth', -1)  # or 199

#check out https://github.com/gershonc/octopus-ml
import octopus_ml as oc

In [ ]:
train_df = pd.read_csv ( "../input/tabular-playground-series-mar-2021/train.csv")
test_df = pd.read_csv("../input/tabular-playground-series-mar-2021/test.csv")


## EDA

In [ ]:
# Data shape 
print ("Train set: ",train_df.shape)
print ("Test set: ",test_df.shape)

In [ ]:
# DataFrane Summary by pandas summary package (extension of pandas.describe method) 
dfs = DataFrameSummary(train_df)
dfs.summary()

In [ ]:
# Top 10 sparse features, mainly labs results 
pd.Series(1 - train_df.count() / len(train_df)).sort_values(ascending=False).head(10)

In [ ]:
# Categorical features

categorical_features=[]
for c in train_df.columns:
    col_type = train_df[c].dtype
    if col_type == 'object' or col_type.name == 'category':
        train_df[c] = train_df[c].astype('category')
        categorical_features.append(c)
print (categorical_features)

In [ ]:
# Target distribution analysis
fig, ax =plt.subplots(1,2)


plt.style.use('fivethirtyeight')
plt.figure(figsize=(3,4))
sns.set_context("paper", font_scale=1.2)                                                  
sns.countplot('target',data=train_df, ax=ax[0])
train_df['target'].value_counts().plot.pie(explode=[0,0.2],autopct='%1.2f%%',ax=ax[1])
fig.show()

In [ ]:
sns.displot(data = train_df, kind = 'hist', x = 'cont1', hue = 'target', multiple = 'stack',bins=25,height = 4, aspect = 1.7)

In [ ]:
sns.displot(data = train_df, kind = 'hist', x = 'cont2', hue = 'target', multiple = 'stack',bins=25,height = 4, aspect = 1.7)

## Data pre-processing


In [ ]:
features=train_df.columns.to_list()
print ('Number of features ', len(features))

In [ ]:
features_remove=['target']
for f in features_remove:
    features.remove(f)

In [ ]:
X=train_df[features]
y=train_df['target']

## HPO (Hyper-parameter Tuning)

In [ ]:
%%time

from bayes_opt import BayesianOptimization


def bayes_parameter_opt_lgb(X, y, init_round=15, opt_round=25, n_folds=6, verbose=-1, random_seed=6, n_estimators=10, output_process=False):
    # prepare data
    train_data = lgb.Dataset(data=X, label=y, free_raw_data=False)
    # parameters
    def lgb_eval(learning_rate,num_leaves, feature_fraction, bagging_fraction, max_depth, max_bin, min_data_in_leaf,min_sum_hessian_in_leaf,subsample,scale_pos_weight):
        params = {'application':'binary', 'metric':'auc'}
        params['learning_rate'] = max(min(learning_rate, 1), 0)
        params["num_leaves"] = int(round(num_leaves))
        params['feature_fraction'] = max(min(feature_fraction, 1), 0)
        params['bagging_fraction'] = max(min(bagging_fraction, 1), 0)
        params['max_depth'] = int(round(max_depth))
        params['max_bin'] = int(round(max_depth))
        params['verbose']=-1
        params['min_data_in_leaf'] = int(round(min_data_in_leaf))
        params['min_sum_hessian_in_leaf'] = min_sum_hessian_in_leaf
        params['subsample'] = max(min(subsample, 1), 0)
        params['scale_pos_weight']=scale_pos_weight
        
        cv_result = lgb.cv(params, train_data, nfold=n_folds, seed=random_seed, stratified=True, verbose_eval =200, metrics=['auc'])
        return np.mean(cv_result['auc-mean'])
     
    lgbBO = BayesianOptimization(lgb_eval, {'learning_rate': (0.01, 0.15),
                                            'num_leaves': (1, 500),
                                            'feature_fraction': (0.1, 0.9),
                                            'bagging_fraction': (0.5, 1),
                                            'max_depth': (2, 20),
                                            'max_bin':(10,30),
                                            'min_data_in_leaf': (20, 80),
                                            'min_sum_hessian_in_leaf':(0,100),
                                            'subsample': (0.01, 1.0),
                                            'scale_pos_weight': (0.1,10)
                                           }, 
                                             random_state=200)

    
    #n_iter: How many steps of bayesian optimization you want to perform. The more steps the more likely to find a good maximum you are.
    #init_points: How many steps of random exploration you want to perform. Random exploration can help by diversifying the exploration space.
    
    lgbBO.maximize(init_points=init_round, n_iter=opt_round)
    
    model_auc=[]
    for model in range(len( lgbBO.res)):
        model_auc.append(lgbBO.res[model]['target'])
    
    # return best parameters
    return lgbBO.res[pd.Series(model_auc).idxmax()]['target'],lgbBO.res[pd.Series(model_auc).idxmax()]['params']

opt_params = bayes_parameter_opt_lgb(X, y, init_round=5, opt_round=150, n_folds=5, random_seed=6,n_estimators=300)

In [ ]:
opt_params[1]["num_leaves"] = int(round(opt_params[1]["num_leaves"]))
opt_params[1]['max_depth'] = int(round(opt_params[1]['max_depth']))
opt_params[1]['min_data_in_leaf'] = int(round(opt_params[1]['min_data_in_leaf']))
opt_params[1]['max_bin'] = int(round(opt_params[1]['max_bin']))
opt_params[1]['objective']='binary'
opt_params[1]['metric']='auc'
#opt_params[1]['is_unbalance']=True
opt_params[1]['boost_from_average']=False
opt_params=opt_params[1]
opt_params

In [ ]:
params = {
        'boosting_type': 'gbdt',
        'objective': 'binary',
        'metric': 'auc',
        'learning_rate': 0.01,
        'subsample': 1,
        'colsample_bytree': 0.2,
        'reg_alpha': 3,
        'reg_lambda': 1,
        'scale_pos_weight': 4,
        'n_estimators': 1000,
        'verbose': 1,
        'max_depth': -1,
        'seed':100, 
        'force_col_wise': True

}

#params.update(opt_params)


clf,arr_f1_weighted,arr_f1_macro,arr_f1_positive,prediction_folds,preds_folds,y_folds= oc.cv(X,y,0.5,1000,shuffle=True,params=params)

In [ ]:
oc.cv_plot(arr_f1_weighted,arr_f1_macro,arr_f1_positive,'TBS match 2021 - Kaggle compatition')


In [ ]:
print(classification_report(y_folds, prediction_folds))


In [ ]:
oc.roc_curve_plot(y_folds,preds_folds)


In [ ]:
feature_imp_list=oc.plot_imp(clf,X,'LightGBM Mortality Kaggle',num=20)


In [ ]:
top_features=feature_imp_list.sort_values(by='Value', ascending=False).head(20)
top_features

In [ ]:
list_for_correlations=top_features['Feature'].to_list()
list_for_correlations.append('target')
oc.correlations(train_df,list_for_correlations)

In [ ]:
def Kaggle_submission(file_name,model,test_data,ids_list):
    if TARGET in test_data.columns:
        test_data.drop([TARGET],axis=1,inplace=True)
    #test_pred=model.predict(test_data[features])[:,1]
    test_pred=model.predict(test_data[features])
    print (test_pred[1:2])

    submit=pd.DataFrame()
    submit['id'] = ids_list
    submit['target'] = test_pred
    submit.to_csv(file_name,index=False)
    return submit

In [ ]:
# Categorical features on testset

categorical_features=[]
for c in test_df.columns:
    col_type = train_df[c].dtype
    if col_type == 'object' or col_type.name == 'category':
        test_df[c] = test_df[c].astype('category')
        categorical_features.append(c)
print (categorical_features)

TARGET="diabetes_mellitus"
submit=Kaggle_submission("LGBM_baseline_v15.csv",clf,test_df,test_df['id'].tolist())